### Loading the required modules.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
import numpy as np
import plotly

### Loading Data into a DataFrame using pandas

In [2]:
file = 'https://raw.githubusercontent.com/varadan13/Data_Bank/main/Kerala%20Assembly%20Election%20Analysis/Detailed-Results-2016-election.csv'

In [3]:
df = pd.read_csv(file)

In [4]:
constituency_column = df.columns[1]  

### Using widgets to get constituency value

In [5]:
constituency = widgets.Select(  # Using select Ipywidgets to change the value of constituency variable 
    options=df[constituency_column].unique(),
    value='MANJESHWAR',
    description='Constituency:',
)
output2 = widgets.Output()

display(constituency, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

constituency.observe(on_value_change, names='value')

Select(description='Constituency:', options=('MANJESHWAR', 'KASARAGOD', 'UDMA', 'KANHANGAD', 'TRIKARIPUR', 'PA…

Output()

In [6]:
constituency_val = constituency.value # extracting the value of constituency in the widget. 

### Visualising the margin of win. 

In [7]:
def margin():  # function which returns the margin of win
  df_2_plot = df[df[constituency_column]==constituency_val]
  temp_1 = int((df_2_plot.iloc[0:1])[df_2_plot.columns[9]])
  temp_2 = int((df_2_plot.iloc[1:2])[df_2_plot.columns[9]])
  return (temp_1 - temp_2)

In [8]:
df_2_plot = df[df[constituency_column]==constituency_val]

In [9]:
fig = go.Figure(go.Indicator(
    mode = "number",
    value = margin(),
   ))

fig.update_layout(title_text='Margin Of Win', paper_bgcolor = "lightgray")
fig.show()

In [10]:
def party_details():
  party_won = np.array((df_2_plot.iloc[0:1])[df_2_plot.columns[6]])[0]
  first_runner_up = np.array((df_2_plot.iloc[1:2])[df_2_plot.columns[6]])[0]
  second_runner_up = np.array((df_2_plot.iloc[2:3])[df_2_plot.columns[6]])[0]
  return (party_won,first_runner_up,second_runner_up)

In [11]:
party_details()

('CPM', 'INC', 'BJP')

### Visualising Election Results

In [12]:
fig = px.bar(df_2_plot.iloc[0:3], x='Candidate Name', y=' Total Valid Votes',
             hover_data=[' Party Name', 'Candidate Sex', 'Candidate Category'],
             color=' Total Valid Votes', title= 'Election Results of '+constituency_val+' Constituency',
             height=400)
fig.show()

In [13]:
def first_place(party):
  temp = 0 
  for i in df[constituency_column].unique():
    df_temp = df[df[constituency_column]==i]
    if np.array((df_temp.iloc[0:1])[df_temp.columns[6]])[0] == party:
      temp += 1
  
  return temp

def third_place(party):
  temp = 0 
  for i in df[constituency_column].unique():
    df_temp = df[df[constituency_column]==i]
    if np.array((df_temp.iloc[2:3])[df_temp.columns[6]])[0] == party:
      temp += 1
  
  return temp

In [14]:
first_win_summary = dict()
for i in df[' Party Name'].unique():
  if first_place(i)>10:
    first_win_summary[i] = [first_place(i)]

first_win_summary['BJP'] = [1]

third_runner_up_summary = dict()
for i in df[' Party Name'].unique():
  if third_place(i) != 0:
    third_runner_up_summary[i] = [third_place(i)]


In [15]:
party_win = pd.DataFrame(first_win_summary).T
party_win = party_win.reset_index()
party_win = party_win.rename(columns={'index':"Party", 0:'wins'})
party_win

,Party,wins
0,IUML,18
1,CPM,58
2,INC,22
3,CPI,19
4,BJP,1


In [16]:
fig = px.bar(party_win, x='Party', y='wins', color='Party', height=400, title = 'Wins Of Major Parties')
fig.show()

In [17]:
third_runner_ups = pd.DataFrame(third_runner_up_summary).T
third_runner_ups = third_runner_ups.reset_index()
third_runner_ups = third_runner_ups.rename(columns={'index':"Party", 0:'wins'})
third_runner_ups

,Party,wins
0,BJP,88
1,CPM,3
2,IND,5
3,INL,1
4,INC,3
5,BDJS,35
6,JD(U),1
7,ADMK,1
8,KEC,3


In [18]:
fig = px.bar(third_runner_ups, x='Party', y='wins', color='Party', height=400, title = 'Third Runner Ups')
fig.show()

### Conclusion  
- I want to implement this visualisation in dash.  
- I must collect demographic data and find correlation between demography and party win for each constituency. 